In [1]:
%load_ext autoreload
%autoreload 2

from kasearch import AlignSequences, SearchDB, PrepareDB

## Prepare query sequence (sequence to search with)

In [2]:
raw_queries = [
    'QVKLQESGAELARPGASVKLSCKASGYTFTNYWMQWVKQRPGQGLDWIGAIYPGDGNTRYTHKFKGKATLTADKSSSTAYMQLSSLASEDSGVYYCARGEGNYAWFAYWGQGTTVTVSS',
]

In [3]:
query_db = AlignSequences(raw_queries, # Sequences as strings to align.
                          n_jobs=1     # Allocated number for jobs/threads for the search.
                         )
query_db.db.aligned_seqs

array([[81, 86, 75,  0, 76, 81, 69, 83, 71, 65,  0, 69, 76, 65, 82, 80,
        71, 65, 83, 86, 75, 76, 83, 67, 75, 65, 83, 71, 89, 84, 70,  0,
         0,  0,  0,  0,  0,  0,  0,  0, 84, 78, 89, 87, 77, 81,  0, 87,
        86, 75, 81,  0, 82,  0, 80,  0, 71,  0, 81,  0,  0, 71,  0, 76,
        68,  0, 87, 73, 71, 65, 73, 89, 80, 71,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0, 68, 71, 78, 84, 82, 89,  0,  0, 84,  0,  0,
        72,  0,  0, 75, 70,  0,  0, 75,  0,  0,  0, 71, 75, 65, 84, 76,
        84, 65,  0, 68,  0,  0,  0, 75,  0, 83,  0,  0, 83, 83,  0,  0,
         0,  0, 84,  0, 65, 89, 77, 81, 76, 83, 83, 76, 65, 83,  0, 69,
        68, 83, 71, 86, 89, 89, 67, 65, 82, 71, 69, 71, 78,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0, 89, 65, 87, 70, 65, 89, 87, 71,  0, 81,
        71, 84, 84, 86, 84, 86, 83, 83]], dtype=int8)

--------------
## Initiate search class 

- Default will download a prepared version of OAS to search against
- Alternatively, you can give it the path for a custom database to search against. (See below for how to create a custom database).

In [4]:
oasdb = SearchDB(database_path='oasdb-small', # Path to your database. Default will be to download a small prepared version of OAS.
                 allowed_chain='Heavy',         # Search against a specific chain. Default is any chain.
                 allowed_species='Human',       # Search against a specific species. Default is any species.
                )

-----------
## Run search

A search against all of OAS takes ~2min for each sequence.


In [5]:
%%time 
oasdb.search(query_db.db.aligned_seqs, # Input can be a single or multiple aligned sequences at a time.
             keep_best_n=10,               # You can define how many most similar sequences to return
            )

CPU times: user 2min 40s, sys: 12.7 s, total: 2min 52s
Wall time: 1min 5s


### Get N best identities

Identities of the most similar sequence (1st column), CDRs (2nd column) and CDR3 (3rd column) in all of OAS, can be fetched from the object with the bellow command.

In [6]:
oasdb.current_best_identities

array([[[0.90378153, 0.78571427, 0.8333333 ],
        [0.8983696 , 0.75      , 0.8333333 ],
        [0.89311516, 0.75      , 0.8333333 ],
        [0.8925946 , 0.75      , 0.8333333 ],
        [0.8907563 , 0.71428573, 0.8333333 ],
        [0.8898944 , 0.71428573, 0.75      ],
        [0.88852537, 0.71428573, 0.75      ],
        [0.8852507 , 0.71428573, 0.75      ],
        [0.88392854, 0.71428573, 0.75      ],
        [0.88235295, 0.71428573, 0.75      ]]], dtype=float32)

### Get ID's of sequences with the N best identities

ID's of the most similar sequence (1st column), CDRs (2nd column) and CDR3 (3rd column) in all of OAS, can be fetched from the object with the bellow command. 

With ExtractMetadata, these ID's can be used to extract the sequence and its meta data (species, isotype, whole sequence etc.).

In [7]:
oasdb.current_best_ids

array([[[[   3595,    3111],
         [   3611,    2327],
         [   3580,    1736]],

        [[   3595,     541],
         [   3611,     396],
         [   3611,    2027]],

        [[   3595,    1427],
         [   3611,    5859],
         [   3574,   20140]],

        [[   3595,     765],
         [   3574,    2556],
         [   3595,     904]],

        [[   3595,     943],
         [   3608,    2991],
         [   3611,     391]],

        [[   3595,    5205],
         [   9423,    2660],
         [  10012,   61994]],

        [[   3629,    1132],
         [  10496,  646471],
         [   3358, 1405904]],

        [[   3629,    1391],
         [  12078,   19355],
         [  10012, 1383316]],

        [[   3595,     923],
         [   9492,  169374],
         [   8614,   74371]],

        [[   3595,   10782],
         [   5262,   30055],
         [   3358,  178333]]]], dtype=int32)

---------
## Extract the meta data from matched sequences


The meta data for the N best of the different regions, can be extracted with the following n_regions.
- 0: whole sequence
- 1: CDRs
- 2: CDR3

e.g. "oasdb.get_meta\[n_query=0,n_region=0\]\" returns meta data for the N best whole sequence identities for the first query.

NB: The column "sequence_alignment_aa" holds the antibody sequence.

In [8]:
n_best_sequences = oasdb.get_meta(n_query = 0,          # Which query to extract meta data from
                                  n_region = 0,         # Which region to extract meta data from
                                  n_sequences = 'all',  # Number of sequences to extract (default is all, which is keep_best_n)
                                  n_jobs=10             # Allocated number for jobs/threads for the extraction
                                 )
n_best_sequences

,sequence,locus,stop_codon,vj_in_frame,v_frameshift,productive,rev_comp,complete_vdj,v_call,d_call,...,Longitudinal,Age,Disease,Subject,Vaccine,Chain,Unique sequences,Total sequences,Isotype,Identity
0,GAAACAACCTATGATCAGTGTCCTCTCTACACAGTCCCTGACGACA...,H,F,T,F,T,F,F,IGHV1-46*01,IGHD3-16*01,...,no,no,POEMS,Patient_12,None,Heavy,21060,37905,Bulk,0.903782
1,GGCATATGATCAGTAACCTCTTCACAGTCACTGAAAACACTGACTC...,H,F,T,F,T,F,F,IGHV1-46*01,IGHD5-12*01,...,no,no,POEMS,Patient_12,None,Heavy,21060,37905,Bulk,0.898370
2,GGCATATGATCAGTAACCTCTTCACAGTCACTGAAAACACTGACTC...,H,F,T,F,T,F,F,IGHV1-46*01,IGHD3-9*01,...,no,no,POEMS,Patient_12,None,Heavy,21060,37905,Bulk,0.893115
3,GACAGTCACTGAAAACACTGACTCTAATCATGGAATGTAACTGGAT...,H,F,T,F,T,F,T,IGHV1-46*01,IGHD5-12*01,...,no,no,POEMS,Patient_12,None,Heavy,21060,37905,Bulk,0.892595
4,GGCATATGATCAGTAACCTCTTCACAGTCACTGAAAACACTGACTC...,H,F,T,F,T,F,F,IGHV1-46*01,IGHD3-3*01,...,no,no,POEMS,Patient_12,None,Heavy,21060,37905,Bulk,0.890756
5,GGCATATGATCAGTAACCTCTTCACAGTCACTGAAAACACTGACTC...,H,F,T,F,T,F,F,IGHV1-46*01,IGHD3-16*01,...,no,no,POEMS,Patient_12,None,Heavy,21060,37905,Bulk,0.889894
6,GGGCATATGATCAGTAACCTCTTCACAGTCACTGAAAACACTGACT...,H,F,T,F,T,T,T,IGHV1-46*01,NaN,...,no,no,POEMS,Patient_26,None,Heavy,31495,34138,Bulk,0.888525
7,GACAGTCACTGAAAACACTGACTCTAATCATGGAATGTAACTGGAT...,H,F,T,F,T,T,T,IGHV1-46*01,IGHD3-16*01,...,no,no,POEMS,Patient_26,None,Heavy,31495,34138,Bulk,0.885251
8,GACAGTCACTGAAAACACTGACTCTAATCATGGAATGTAACTGGAT...,H,F,T,F,T,F,F,IGHV1-46*01,IGHD3-16*01,...,no,no,POEMS,Patient_12,None,Heavy,21060,37905,Bulk,0.883929
9,GACCGCATATGATCAGTAACCTCTTCACAGTCACTGAAAACACTGA...,H,F,T,F,T,F,T,IGHV1-46*01,IGHD3-10*01,...,no,no,POEMS,Patient_12,None,Heavy,21060,37905,Bulk,0.882353


In [9]:
n_best_sequences.sequence_alignment_aa.values

array(['QVQLQQSGAELARPGASVKLSCKASGYTFTSYWMQWVKQRPGQGLEWIGAIYPGDGDTRYTQKFKGKATLTADKSSSTAYMQLSSLASEDSAVYYCARGEPRYDYAWFAYWGQGTLVTVS',
       'QVQLQQSGAELARPGASVKLSCKASGYTFTSYWMQWVKQRPGQGLEWIGAIYPGDGDTRYTQKFKGKATLTADKSSSTAYMQLSSLASEDSAVYYCARGPATAWFAYWGQGTLVTVS',
       'QVQLQQSGAELARPGASVKLSCKASGYTFTSYWMQWVKQRPGQGLEWIGAIYPGDGDTRYTQKFKGKATLTADKSSSTAYMQLSSLASEDSAVYYCARSAWFAYWGQGTLVTVS',
       'QVQLQQSGAELARPGASVKLSCKASGYTFTSYWMQWVKQRPGQGLEWIGAIYPGDGDTRYTQKFKGKATLTADKSSSTAYMQLSSLASEDSAVYYCARGGYWGQGTTLTVSS',
       'QVQLQQSGAELARPGASVKLSCKASGYTFTSYWMQWVKQRPGQGLEWIGAIYPGDGDTRYTQKFKGKATLTADKSSSTAYMQLSSLASEDSAVYYCARGGLRRGAWFAYWGQGTLVTVS',
       'QVQLQQSGAELARPGASVKLSCKASGYTFTSYWMQWVKQRPGQGLEWIGAIYPGDGDTRYTQKFKGKATLTADKSSSTAYMQLSSLASEDSAVYYCASGDYDAWFAYWGQGTLVTVS',
       'QVQLQQSGAELARPGASVKLSCKASGYTFTSYWMQWVKQRPGQGLEWIGAIYPGDGDTRYTQKFKGKATLTADKSSSTAYMQLSSLASEDSAVYYCAKGFDYWGQGTTLTVSS',
       'QVQLQQSGAELARPGASVKLSCKASGYTFTSYWMQWVKQRPGQGLEWIGAIYPGDGDTRYTQKFKGKATLTADKSSSTAYMQLSSLASEDSAVYYCARGDSWF

----------
## Create custom database


To create your own database you first need to create a csv file in the OAS format. For an example file, look at data/custom-data-example.csv. This file consists of a dictionary containing the metadata in the first line and then rows of the individual sequences afterwards. Only the Species and Chain is strictly needed in the metadata, and only the amino acids sequence of the antibodies is required for each antibody sequence.

### 1. Format your data into OAS files

In [10]:
import json
import os
import pandas as pd

from kasearch.species_anarci import number
from kasearch.merge_db import merge_files

In [11]:
metadata = {"Species":"Human", "Chain":"Heavy"}
metadata = pd.Series(name=json.dumps(metadata), dtype='object')
seqsdata = pd.DataFrame([["EVQLVESGGGLAKPGGSLRLHCAASGFAFSSYWMNWVRQAPGKRLEWVSAINLGGGLTYYAASVKGRFTISRDNSKNTLSLQMNSLRAEDTAVYYCATDYCSSTYCSPVGDYWGQGVLVTVSS"],
                          ["EVQLVQSGAEVKRPGESLKISCKTSGYSFTSYWISWVRQMPGKGLEWMGAIDPSDSDTRYNPSFQGQVTISADKSISTAYLQWSRLKASDTATYYCAIKKYCTGSGCRRWYFDLWGPGT"]
                         ], columns = ['sequence_alignment_aa'])

In [12]:
save_file = "../data/custom-data-examples.csv"
metadata.to_csv(save_file, index=False)
seqsdata.to_csv(save_file, index=False, mode='a')

### 2. Turn your OAS formatted files into a custom database

After creating all the files you want to include in the new database, you can run the following code to create the database.

In [13]:
db_folder = "../data/my_db"
db_files = ['../data/custom-data-examples.csv']

In [14]:
newDB = PrepareDB(db_path=db_folder)

db_dict = {}
for num, data_unit_file in enumerate(db_files):

    db_dict[num] = data_unit_file
    
    metadata = json.loads(','.join(pd.read_csv(data_unit_file, nrows=0).columns))
    seqsdata = pd.read_csv(data_unit_file, header=1, usecols=['sequence_alignment_aa']).iloc[:,0].values
    numbered_sequences = [number(sequence, allowed_species=None)[0] for sequence in seqsdata]

    newDB.prepare_database(numbered_sequences, 
                           file_id=num, 
                           chain=metadata['Chain'], 
                           species=metadata['Species'])
    
newDB.save_database()

merge_files(db_folder)

with open(os.path.join(db_folder, "id_to_study.txt"), "w") as handle: handle.write(str(db_dict))

### 3. Initiate the search class with your custom database


In [15]:
mydb = SearchDB(database_path=db_folder,      # Path to your database. Default will be to download a prepared version of OAS.
                 allowed_chain='Heavy',       # Search against a specific chain. Default is any chain.
                 allowed_species='Any',       # Search against a specific species. Default is any species.
                )